In [ ]:
import pandas as pd
import pickle
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#Step 1: Loading the saved model and scaler from pkl files

with open("../artifacts/xgboost_model.pkl", "rb") as file:
    loaded_model = pickle.load(file)

#Step 2: Loading the test dataset
df_test = pd.read_csv('../data/test_data.csv', delimiter=",")

#Step 3: Preparing the test data
X_test = df_test.drop(columns=["StudentID", "GradeClass", "GPA"])
y_test = df_test['GradeClass']

#Step 4: Make Predictions
predictions = loaded_model.predict(X_test)

#Step 5: Prediction function
def predict_student_grade(input_string):
    columns = ["Age", "Gender", "Ethnicity", "ParentalEducation", "StudyTimeWeekly", "Absences", "Tutoring", "ParentalSupport", "Extracurricular","Sports", "Music", "Volunteering"]

    # Convert input string into a list
    input_data = input_string.split(",")

    # Automatically remove the first value (StudentID) and last two values (GPA & GradeClass)
    input_data = input_data[1:-2] # Removes 1 from start and last 2 items

    # Convert to DataFrame
    df_input = pd.DataFrame([input_data], columns=columns)

    # Create DataFrame from input
    df_input = pd.DataFrame([input_data], columns=columns)
    df_input = df_input.apply(pd.to_numeric, errors='ignore')  # Cast numerics properly

    # Predict using the loaded model
    prediction = loaded_model.predict(df_input)[0]
    
    #Mapping numeric class baack to the letter grade
    grade_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "F"}

    predicted_grade = grade_map[prediction] 

    return predicted_grade

#Step 6: Evaluate predictions
print("Accuracy: ", accuracy_score(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions, target_names=["A", "B", "C", "D", "F"]))

#Example testing input from the csv file
student_input = "1045,18,1,0,1,18.921512361376628,1,1,3,1,1,0,0,4.0,0.0"

#Print predicted grade
print(f"Predicted Grade: {predict_student_grade(student_input)} ,for given student input")

#Confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix(y_test, predictions), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()